# Past Values and Past Errors: The ARMA Model

* Limitations of AR or MA models 

  * Both AR and MA have residuals that is not white noise. This means neither of them is suitable for fitting price return data. 
  * AR limitation: fail to adjust quickly to unexpected shocks --> need MA aspects to smooth out prediction
  * MA limitation:  need a baseline to perform well --> need to use previous values (AR) 
  * AR + MA: solves the issues each one has individually 

* Definition 

  * Equation ARMA(1) 
    $$
    r_t = c + \varphi_1r_{t-1} + \theta_1\epsilon_{t-1} + \epsilon_t
    $$

    * $r_t$, $r_{t-1}$: The values of "r" in the current period and 1 period ago respectively
    * $\epsilon_t$, $\epsilon_{t-1}$ : Residuals for the current period and the 1 period ago respectively
    * $c$: Baseline constant factor 
    * $\varphi_1$: The part of the value last period is relevant in explaining the current one, $|\varphi_n|<1$ to prevent compounded effects exploding in magnitude
    * $\theta_1$: The part of the value last period is relevant in explaining the current one, $|\theta_n|<1$ to prevent compounded effects exploding in magnitude 

  * ARMA(p, q)

    * p: number of lagged values (AR order)
    * q: number of lagged errors (MA order)
    * p and q don't have to be the same value  
    * Actual - prediction -> how foar off our predictions were -> calibrate expections on the go 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as sts
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima_model import ARMA
from scipy.stats.distributions import chi2
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data = pd.read_csv('../data/Index2018.csv')
df_comp = raw_data.copy()
df_comp.date = pd.to_datetime(df_comp.date, dayfirst=True)
df_comp.set_index("date", inplace=True)
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method='ffill')

In [3]:
df_comp['market_value'] = df_comp.ftse
del df_comp['spx']
del df_comp['dax']
del df_comp['ftse']
del df_comp['nikkei']

size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size].copy(), df_comp.iloc[size:].copy()

In [4]:
df['returns'] = df.market_value.pct_change(1).mul(100)

In [5]:
def LLR_test(mod_1, mod_2, DF=1):
    # DF: degrees of freedom
    
    # log likelihood
    L1 = mod_1.fit().llf
    L2 = mod_2.fit().llf
    
    # test statistic
    LR = (2 * (L2 - L1))
    p = chi2.sf(LR, DF).round(3)
    
    return p

# ARMA(1, 1)

In [6]:
model_ret_ar_1_ma_1 = ARMA(df.returns[1:], order=(1,1))
results_ret_ar_1_ma_1 = model_ret_ar_1_ma_1.fit()
results_ret_ar_1_ma_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                returns   No. Observations:                 5020
Model:                     ARMA(1, 1)   Log Likelihood               -7916.500
Method:                       css-mle   S.D. of innovations              1.171
Date:                Thu, 28 Jan 2021   AIC                          15841.000
Time:                        23:22:31   BIC                          15867.085
Sample:                    01-10-1994   HQIC                         15850.140
                         - 04-05-2013                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0189      0.013      1.446      0.148      -0.007       0.045
ar.L1.returns     0.7649      0.067     11.349      0.000       0.633       0.897
ma.L1.returns    -0.8141      0.061    -13.406      0.000      -0.933      -0.695
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.3074           +0.0000j            1.3074            0.0000
MA.1            1.2284           +0.0000j            1.2284            0.0000
-----------------------------------------------------------------------------
"""

The p-value for constant is not significantly different from zero. As we are using returns, it is within reason for returns to be centered around zero. This is completely acceptible. 

The coefficient for past value is over 0.5 meaning there is a positive tendency between past and present values. In other words, returns move in trends of consecutive positive or negative. When translating it into prices, prices tends to consistently increase or decrease. 

The negative coefficient for the past error is slightly harder to interpret. We should be moving away from the past period (t-1) values. These pas error terms ensure we don't get a "fool in the shower" type of error.   

We want to compare the ARMA(1, 1) with the AR(1) and MA(1) models using LLR test because they are nested in the ARMA(1, 1).

In [7]:
# LLR Test 
model_ret_ar_1 = ARMA(df.returns[1:], order=(1,0))
model_ret_ma_1 = ARMA(df.returns[1:], order=(0,1))


print("ARMA vs AR", LLR_test(model_ret_ar_1, model_ret_ar_1_ma_1))
print("ARMA vs MA", LLR_test(model_ret_ma_1, model_ret_ar_1_ma_1))

ARMA vs AR 0.0
ARMA vs MA 0.0


This verifies that ARMA(1,1) is better than AR(1) and MA(1).

# Higher-Lag ARMA Models

Check ACF & PACF. They suggests AR(8) and MA(6). We could include all the important lags but it can be redundant and it might overfit. We could start with ARMA(3, 3) which is about half of the first guess.

In [8]:
model_ret_ar_3_ma_3 = ARMA(df.returns[1:], order=(3,3))
results_ret_ar_3_ma_3 = model_ret_ar_3_ma_3.fit()

In [9]:
results_ret_ar_3_ma_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                returns   No. Observations:                 5020
Model:                     ARMA(3, 3)   Log Likelihood               -7893.515
Method:                       css-mle   S.D. of innovations              1.166
Date:                Thu, 28 Jan 2021   AIC                          15803.030
Time:                        23:22:37   BIC                          15855.199
Sample:                    01-10-1994   HQIC                         15821.310
                         - 04-05-2013                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0189      0.014      1.395      0.163      -0.008       0.045
ar.L1.returns    -0.1898      0.104     -1.827      0.068      -0.393       0.014
ar.L2.returns    -0.2942      0.087     -3.389      0.001      -0.464      -0.124
ar.L3.returns     0.4459      0.138      3.225      0.001       0.175       0.717
ma.L1.returns     0.1707      0.099      1.726      0.084      -0.023       0.365
ma.L2.returns     0.2277      0.084      2.701      0.007       0.062       0.393
ma.L3.returns    -0.5432      0.127     -4.270      0.000      -0.793      -0.294
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -0.5168           -1.0283j            1.1508           -0.3241
AR.2           -0.5168           +1.0283j            1.1508            0.3241
AR.3            1.6932           -0.0000j            1.6932           -0.0000
MA.1           -0.5286           -0.9835j            1.1166           -0.3285
MA.2           -0.5286           +0.9835j            1.1166            0.3285
MA.3            1.4764           -0.0000j            1.4764           -0.0000
-----------------------------------------------------------------------------
"""

The first coefficients for AR and MA are not significant.

Is this better than ARMA(1,1)?

In [10]:
print("ARMA(1,1) vs ARMA(3,3):", LLR_test(model_ret_ar_1_ma_1, model_ret_ar_3_ma_3, 4))

ARMA(1,1) vs ARMA(3,3): 0.0


ARMA(3, 3) is better than ARMA(1, 1). But is ARMA(3, 3) optimal? The best fit would lie between ARMA(1, 1) and ARMA(3, 3). How can we find the best one though?

# ARMA(3, 2)

We still want a complicated model. By reducing the order by 1 from ARMA(3, 3), we begin with the models that consist of five total orders. ARMA(3, 2) and ARMA(2, 3). It makes little difference which one we try first. So, let's start with ARMA(3, 2).

In [15]:
model_ret_ar_3_ma_2 = ARMA(df.returns[1:], order=(3,2))
results_ret_ar_3_ma_2 = model_ret_ar_3_ma_2.fit()
results_ret_ar_3_ma_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                returns   No. Observations:                 5020
Model:                     ARMA(3, 2)   Log Likelihood               -7895.747
Method:                       css-mle   S.D. of innovations              1.166
Date:                Thu, 28 Jan 2021   AIC                          15805.495
Time:                        23:29:06   BIC                          15851.143
Sample:                    01-10-1994   HQIC                         15821.491
                         - 04-05-2013                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0188      0.015      1.251      0.211      -0.011       0.048
ar.L1.returns    -0.6785      0.087     -7.799      0.000      -0.849      -0.508
ar.L2.returns    -0.5088      0.139     -3.670      0.000      -0.780      -0.237
ar.L3.returns    -0.1141      0.015     -7.655      0.000      -0.143      -0.085
ma.L1.returns     0.6568      0.087      7.536      0.000       0.486       0.828
ma.L2.returns     0.4474      0.141      3.175      0.001       0.171       0.724
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -0.4929           -1.5101j            1.5885           -0.3002
AR.2           -0.4929           +1.5101j            1.5885            0.3002
AR.3           -3.4748           -0.0000j            3.4748           -0.5000
MA.1           -0.7340           -1.3025j            1.4951           -0.3317
MA.2           -0.7340           +1.3025j            1.4951            0.3317
-----------------------------------------------------------------------------
"""

All of the p-values are significant except for the constant, but the returns in an efficient market should be close to 0 anyway so it shouldn't bother us. 

The absolute value of the coefficients decreases for the higher lags. This supports the idea that the further back in time we go, the less relevant values and errors become. It doesn't have to be the way always, but such a trend makes our predictions seem much more realistic.

Plus, positive MA coefficients suggest calibration efforts. If our prediction was lower than the actual value, we get a positive residual value (error = actual - predicted). With the positive error and positive coefficient, we get a positive MA component. ($\epsilon_t > 0$ ==> $\theta\epsilon_t > 0$) This results in an increase in the value of our predictions for the next period. So we try to close the gap to the actual value hoping the pattern will translate into the future. On the other hand, if the prediction was higher than the actual, the MA component will be negative (negative residual * positive coefficient). This will decrease the prediction for the next period. 

What about the negative AR coefficients? Those more or less match our expectations of an efficient market. If returns are positive, then multiplying the values by a negative factor will move thir effect in the opposite direction. We expect an efficient market to have a mean of zero over time. Therefore, every period of positive returns is followed by one with negative returns. The assumption is that it allows us to remain close to the mean of zero regardless of the starting and ending periods of our sample.


Also, in this case, we don't have to test LLR test because ARMA(3, 2) satisfies the criteria and is a simpler model. 


# ARMA(2, 3)

In [11]:
model_ret_ar_2_ma_3 = ARMA(df.returns[1:], order=(2,3))
results_ret_ar_2_ma_3 = model_ret_ar_2_ma_3.fit()
results_ret_ar_2_ma_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                returns   No. Observations:                 5020
Model:                     ARMA(2, 3)   Log Likelihood               -7895.587
Method:                       css-mle   S.D. of innovations              1.166
Date:                Thu, 28 Jan 2021   AIC                          15805.174
Time:                        23:23:34   BIC                          15850.823
Sample:                    01-10-1994   HQIC                         15821.170
                         - 04-05-2013                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0189      0.015      1.276      0.202      -0.010       0.048
ar.L1.returns    -0.5605      0.090     -6.245      0.000      -0.736      -0.385
ar.L2.returns    -0.4187      0.193     -2.172      0.030      -0.797      -0.041
ma.L1.returns     0.5378      0.090      6.000      0.000       0.362       0.714
ma.L2.returns     0.3540      0.195      1.818      0.069      -0.028       0.736
ma.L3.returns    -0.1158      0.016     -7.369      0.000      -0.147      -0.085
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -0.6694           -1.3930j            1.5455           -0.3213
AR.2           -0.6694           +1.3930j            1.5455            0.3213
MA.1           -0.7270           -1.1772j            1.3836           -0.3381
MA.2           -0.7270           +1.1772j            1.3836            0.3381
MA.3            4.5096           -0.0000j            4.5096           -0.0000
-----------------------------------------------------------------------------
"""

We shouldn't use this because of the p-value of MA(2) but we can perform LLR test. 

In [14]:
LLR_test(model_ret_ar_2_ma_3, model_ret_ar_3_ma_3)

0.042

The value suggests the difference is significant but just barely. This indicates we should opt for the ARMA(3, 3) model instead of the ARMA(2, 3) if we had to choose only between these two. 